In [3]:
import sys
!{sys.executable} -m pip install scikit-learn-intelex

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 19.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 21.3 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 MB 4.7 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 27.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 19.5 MB/s eta 0:00:00


In [ ]:
from sklearnex import patch_sklearn
patch_sklearn(global_patch=True)

In [7]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('/home/idies/workspace/Storage/s.hossain18/Permanent/exodata_big.csv').sort_index(axis=0)

In [ ]:
# Fill missing values with median of each column
for column in data.columns:
    data[column].fillna(data[column].median(), inplace=True)

In [4]:
from sklearn.utils import shuffle

In [6]:
data_shuffled = shuffle(data_filled, random_state=1)

NameError: name 'data_filled' is not defined

In [ ]:
# Find columns with only 0 or 1 values
columns_to_drop = []
for column in data_shuffled.columns:
    if column != "exoplanet":
        unique_values = data_shuffled[column].unique()
        if len(unique_values) == 1 or (len(unique_values) == 2 and 0 in unique_values and 1 in unique_values):
            columns_to_drop.append(column)

# Drop the identified columns
df_filtered = data_shuffled.drop(columns=columns_to_drop)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=43,stratify=y)

In [ ]:
# Convert DataFrame to NumPy array
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()
# Save NumPy array as npy file
np.save('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/y_train.npy', y_train)
np.save('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/y_test.npy', y_test)

In [8]:
from sklearn.preprocessing import RobustScaler
rs = RobustScaler()

In [ ]:
X_train = rs.fit_transform(X_train)
X_test = rs.transform(X_test)

In [ ]:
# Save NumPy array as npy file
np.save('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/X_test.npy', X_test)

In [11]:
from sklearn.decomposition import IncrementalPCA
# Define the batch size and number of components
batch_size = 1000
n_components = 200

In [10]:
# Create an instance of IncrementalPCA
ipca = IncrementalPCA(n_components=n_components)

In [ ]:
# Perform Incremental PCA on the data in batches
for batch_start in range(0, len(X_train), batch_size):
    batch_end = batch_start + batch_size
    batch_data = X_train[batch_start:batch_end]
    ipca.partial_fit(batch_data)

In [ ]:
# Transform the data using the learned components
transformed_data = ipca.transform(X_train)
# Save NumPy array as npy file
np.save('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/X_train_pca.npy', transformed_data)

In [ ]:
# Save DataFrame as Target
#data_array = df_filtered[['exoplanet']].to_numpy()
#df_filtered[['exoplanet']].to_csv('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/data_target.csv', index=False)